In [ ]:
!pip install torch transformers datasets accelerate peft bitsandbytes


In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/.../dataset_half_balanced.json")

In [ ]:
from huggingface_hub import login
login(token='hf_your_token_here')


In [ ]:
from transformers import AutoTokenizer

# Carica il tokenizer pre-addestrato del modello Llama 2 7B in formato Hugging Face.
# Il tokenizer serve a convertire testo in token numerici comprensibili dal modello.
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

# Imposta il token di padding uguale al token di fine sequenza (EOS token).
# Questo è utile perché alcuni modelli non hanno un token di padding dedicato.
tokenizer.pad_token = tokenizer.eos_token


# Funzione per tokenizzare un esempio del dataset
def tokenize(example):
    # Crea un prompt strutturato a partire dai campi del dataset
    # 'instruction', 'input', 'output'. Il prompt ha la forma:
    # ### Instruction:
    # <istruzione>
    # ### Input:
    # <input>
    # ### Response:
    # <output>
    # Questo formato è spesso usato per addestrare modelli instruction-following.
    prompt = f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}"
    
    # Tokenizza il prompt:
    # - truncation=True → tronca il testo se supera max_length
    # - padding="max_length" → aggiunge padding fino a max_length
    # - max_length=512 → lunghezza massima dei token
    # Restituisce un dizionario con input_ids e attention_mask pronto per il modello.
    return tokenizer(prompt, truncation=True, padding="max_length", max_length=512)


# Applica la funzione di tokenizzazione a tutto il dataset.
# `dataset.map()` crea un nuovo dataset in cui ogni esempio è già tokenizzato.
tokenized_dataset = dataset.map(tokenize)


In [ ]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# Carica il modello Llama-2 7B pre-addestrato in modalità Causal Language Modeling.
# load_in_8bit=True → utilizza la quantizzazione a 8 bit per ridurre l'uso di memoria.
# device_map="auto" → assegna automaticamente i layer del modello ai dispositivi disponibili (CPU/GPU).
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    load_in_8bit=True,
    device_map="auto"
)

# Configurazione per il LoRA (Low-Rank Adaptation)
# LoRA permette di fare fine-tuning aggiungendo pochi parametri senza aggiornare tutto il modello.
peft_config = LoraConfig(
    r=8,                        # Rank della matrice di aggiornamento low-rank
    lora_alpha=32,               # Moltiplicatore di scaling per stabilizzare l’addestramento LoRA
    target_modules=["q_proj", "v_proj"],  # Layer del modello dove applicare LoRA (tipicamente Q e V delle attention)
    lora_dropout=0.05,           # Dropout applicato ai pesi LoRA per regolarizzazione
    bias="none",                 # Non aggiunge bias aggiuntivo nel fine-tuning
    task_type=TaskType.CAUSAL_LM # Tipo di task: Causal Language Modeling
)

# Applica LoRA al modello originale.
# Restituisce un modello PEFT che contiene i pesi originali congelati + i parametri LoRA addestrabili.
model = get_peft_model(model, peft_config)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Configurazione dei parametri di training
training_args = TrainingArguments(
    output_dir="./llama-finetuned",  # Cartella dove salvare i checkpoint del modello fine-tuned
    per_device_train_batch_size=2,    # Dimensione del batch per GPU/TPU/CPU
    gradient_accumulation_steps=4,    # Accumula i gradienti per simulare batch più grandi
    num_train_epochs=1,               # Numero di epoche di training sul dataset
    learning_rate=2e-4,               # Learning rate per l'ottimizzatore
    logging_dir="./logs",             # Cartella per salvare i log di training
    logging_steps=10,                 # Frequenza (in step) di scrittura dei log
    save_strategy="epoch",            # Salva il modello alla fine di ogni epoca
    fp16=True                         # Abilita mixed precision (half precision) per ridurre uso memoria e velocizzare training
)

# Collator dei dati per il Language Modeling
# Prepara batch di input per il modello
# mlm=False → modello non usa masked language modeling, adatto a causal LM come Llama
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# Creazione del Trainer di Hugging Face
trainer = Trainer(
    model=model,                      # Modello da allenare
    args=training_args,                # Parametri di training definiti sopra
    train_dataset=tokenized_dataset["train"],  # Dataset di training già tokenizzato
    tokenizer=tokenizer,               # Tokenizer associato al modello
    data_collator=data_collator        # Funzione che prepara i batch durante il training
)

# Avvia il training del modello
trainer.train()


In [ ]:
prompt = """### Instruction:
You are an email security analysis assistant
### Input:
We attempted to deliver your package today, but were unable to complete the delivery due to missing address information.
Please update your delivery details as soon as possible to avoid return of the shipment:
Update Delivery Information
Thank you for your cooperation,
Logistics Service Team
### Response:
"""

enc = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
)

input_ids = enc.input_ids.cuda()
attention_mask = enc.attention_mask.cuda()

outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=1,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
   

In [ ]:
model.save_pretrained("./llama-finetuned")
tokenizer.save_pretrained("./llama-finetuned")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# ----------------------------
# CONFIGURAZIONE PATH E MODELLO
# ----------------------------

BASE_MODEL_NAME = "meta-llama/Llama-2-7b-hf"
LORA_ADAPTER_PATH = "./llama-finetuned"

# ----------------------------
# CARICAMENTO TOKENIZER
# ----------------------------

tokenizer = AutoTokenizer.from_pretrained(LORA_ADAPTER_PATH)

# Per LLaMA è buona pratica assicurarsi che il pad_token sia definito
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ----------------------------
# CARICAMENTO MODELLO BASE
# ----------------------------

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    load_in_8bit=True,        # oppure load_in_4bit=True se usi QLoRA
    device_map="auto"
)

# ----------------------------
# CARICAMENTO ADAPTER LoRA
# ----------------------------

model = PeftModel.from_pretrained(
    base_model,
    LORA_ADAPTER_PATH
)

model.eval()



In [ ]:
def dataset_metrics(df, name="Dataset"):
    
    # Stampa il titolo della sezione con il nome del dataset
    print(f"\n📊 Metriche - {name}")
    
    # Stampa una linea separatrice per migliorare la leggibilità
    print("-" * 50)
    
    # Stampa il numero di righe del DataFrame
    print(f"Numero righe: {df.shape[0]}")
    
    # Stampa il numero di colonne del DataFrame
    print(f"Numero colonne: {df.shape[1]}")
    
    # Elenco delle colonne presenti nel DataFrame
    print("\nColonne:")
    print(df.columns.tolist())
    
    # Conteggio dei valori mancanti (NaN) per ciascuna colonna
    print("\nValori mancanti per colonna:")
    print(df.isnull().sum())
    
    # Visualizzazione del tipo di dato di ogni colonna
    print("\nTipi di dato:")
    print(df.dtypes)


In [ ]:
import pandas as pd
import csv
import sys

CSV_PATH = "TREC_06.csv"  # percorso CSV
csv.field_size_limit(sys.maxsize)

df = pd.read_csv(
    CSV_PATH,
    engine='python',
)

# Stampa le prime righe per verificare
columns_to_drop = ['sender', 'receiver', 'date', 'subject', 'urls']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

In [ ]:
dataset_metrics(df,"Dataset Test con solo body")

In [ ]:
# Stampa il numero totale di righe presenti nel DataFrame originale
print("Numero di righe iniziali:", len(df))

# Stampa la distribuzione iniziale delle etichette (label)
# utile per valutare eventuali sbilanciamenti di classe
print("Conteggio label iniziali:")
print(df['label'].value_counts())

# Rimozione delle righe con valori NaN nelle colonne 'label' o 'body'
# Questo assicura che ogni record abbia un'etichetta valida e un testo associato
df_clean = df.dropna(subset=['label', 'body'])

# Rimozione delle righe in cui il campo 'body' è vuoto o contiene solo spazi
# La funzione str.strip() elimina gli spazi bianchi prima del controllo
df_clean = df_clean[df_clean['body'].str.strip() != '']

# Stampa del numero di righe dopo le operazioni di pulizia
print("\nNumero di righe dopo la pulizia:", len(df_clean))

# Stampa la distribuzione delle etichette dopo la pulizia
# per verificare l'impatto delle operazioni sui dati
print("Conteggio label dopo la pulizia:")
print(df_clean['label'].value_counts())


In [ ]:
# Liste per memorizzare le risposte del modello e le label reali
risposte = []
risposte_vere = []

# Contatori per il calcolo delle metriche
corretti = 0
righe_valide = 0

# Numero totale di righe del dataset
dim = len(df)

# Percorso del file di log dei prompt
prompt_log_path = "prompt_log.txt"

# Apertura del file di testo in modalità scrittura (sovrascrive se esiste)
with open(prompt_log_path, "w", encoding="utf-8") as prompt_file:

    # Iterazione su tutte le righe del DataFrame
    for i in range(0, dim):
        first_row = df.iloc[i]

        # Estrazione del testo dell'email e della label associata
        # Rimozione di spazi e newline finali dal corpo dell'email
        #body_text = str(first_row['body']).rstrip()
        body_text = first_row['body']
        label = first_row['label']

        # Costruzione del prompt passato al modello
        prompt = f"""
### Instruction:
You are an email security analysis assistant
### Input:
{body_text}
### Response:
"""

        # Tokenizzazione del prompt
        enc = tokenizer(
            prompt,
            return_tensors="pt",
            padding=True,
        )

        # Calcolo del numero di token del prompt
        num_tokens = enc.input_ids.shape[1]

        # Scarto delle righe che superano il limite massimo di token
        if num_tokens > 512:
            continue

        # Incremento del numero di righe valide
        righe_valide += 1

        # Spostamento degli input sulla GPU
        input_ids = enc.input_ids.cuda()
        attention_mask = enc.attention_mask.cuda()

        # Generazione della risposta del modello
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=1,
            pad_token_id=tokenizer.eos_token_id
        )

        # Decodifica dell'output in formato testuale
        risposta = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Estrazione della risposta dopo il marker "### Response:"
        risultato = risposta.split("### Response:", 1)[1].strip()

        try:
            # Conversione della risposta in valore numerico
            risultato = float(risultato)

            # Verifica della correttezza con tolleranza ±0.1
            if abs(risultato - label) <= 0.1:
                corretti += 1

        except:
            # Caso di risposta non numerica o malformata
            risultato = "Formato Sbagliato"

        # Salvataggio delle risposte e delle label reali
        risposte.append(risultato)
        risposte_vere.append(label)
"""
        # Scrittura su file DOPO la predizione
        prompt_file.write(
            "\n" + "=" * 100 + "\n"
            f"DATASET INDEX: {i}\n"
            f"LABEL VERA: {label}\n"
            f"LABEL PREDETTA: {risultato}\n"
            + "-" * 100 + "\n"
        )
        prompt_file.write(prompt)
        prompt_file.write("\n")
"""
        # Feedback intermedio ogni 100 righe valide
        if righe_valide % 100 == 0:
            accuracy_temp = corretti / righe_valide if righe_valide > 0 else 0
            formati_sbagliati = risposte.count("Formato Sbagliato")

            print(
                f"[FEEDBACK PARZIALE] "
                f"Righe valide: {righe_valide} | "
                f"Corretti: {corretti} | "
                f"Accuracy: {accuracy_temp:.4f} | "
                f"Formati sbagliati: {formati_sbagliati}"
            )

# Stampa del riepilogo finale
print(
    "*" * 10 +
    "\nEsito: " + str(corretti / righe_valide) +
    "\n\nPredizioni Corrette: " + str(corretti) +
    "\n\nPredizioni Totali: " + str(righe_valide) +
    "\n\nNumero di Formati Sbagliati: " + str(risposte.count("Formato Sbagliato")) +
    "\n" + "*" * 10
)


In [ ]:
def confronta_liste(lista1, lista2):
    # Lista che conterrà gli indici in cui le due liste differiscono
    indici_diversi = []

    # Determina la lunghezza massima tra le due liste
    # per gestire anche il caso di liste di dimensione diversa
    max_len = max(len(lista1), len(lista2))

    # Iterazione su tutti gli indici possibili
    for i in range(max_len):
        # Caso in cui l'indice supera la lunghezza di lista1
        # (elemento mancante in lista1)
        if i >= len(lista1):
            indici_diversi.append(i)

        # Caso in cui l'indice supera la lunghezza di lista2
        # (elemento mancante in lista2)
        elif i >= len(lista2):
            indici_diversi.append(i)

        # Caso in cui entrambi gli elementi esistono ma sono diversi
        elif lista1[i] != lista2[i]:
            indici_diversi.append(i)

    # Restituisce la lista degli indici non coincidenti
    return indici_diversi


# Confronto tra le risposte generate dal modello e le etichette reali
# per individuare le posizioni in cui le predizioni sono errate
Indici = confronta_liste(risposte, risposte_vere)


In [ ]:
# Stampa l'elenco degli indici in cui le predizioni del modello
# differiscono dalle etichette reali
print(Indici)

# Iterazione sugli indici errati per analizzare nel dettaglio
# ciascun errore di classificazione
for indice in Indici:
    # Stampa l'indice della riga, la risposta generata dal modello
    # e la risposta corretta (ground truth)
    print(
        "Indice:", indice,
        "Risposta:", risposte[indice],
        "Risposta Corretta:", risposte_vere[indice]
    )


In [ ]:
prompt = """### Instruction:
You are an email security analysis assistant
### Input:
| As far as I'm concerned this is only a minor irritant -- Caps Lock is
| pointless anyway in these days of OPERATING SYSTEMS THAT DON'T REQUIRE
| YOU TO SHOUT -- but I wondered if anyone else had noticed this bug-ette
| and/or had a fix for it?

It doesn't show up here, running MIT X11.  

In terms of shouting, if you use MODULA-3, encrusted as it is with
upper case keywords, caps-lock is about the only alternative (and a
poor one at that) to a context sensitive editor like emacs.

### Response:
"""

enc = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
)

input_ids = enc.input_ids.cuda()
attention_mask = enc.attention_mask.cuda()

outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=1,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
   